<div >
<img src = "../banner.jpg" />
</div>

# Aplications


-   To illustrate how it works let me use this experiment from the General Social Survey (GSS)

-   GSS conducts surveys regular surveys on Americans think feel about different issues

-   For decades, scholars studying Americans' support for social welfare spending have noted the special disdain that americans harbor for programs labeled "welfare"

-   This phenomenon became the subject of sustained experimental inquiry in the mid-1980s, when the GSS included a question-wording experiment in its national survey of adults.


-   Respondents in each survey were randomly assigned to one of two questions about public spending.

-   *"too much" money is spent on assistance to the Poor (treatment) or Welfare (control)*

-   Various explanations put forward: stereotypes associated with welfare recipients and poor people, particularly racial stereotypes, and to political orientations such as individualism and conservatism .


## Loading packages

Let's load the packages

In [ ]:
# install.packages("pacman") #run this line if you use Google Colab

In [ ]:
set.seed(201911) 

require("pacman")
p_load("tidyverse",# Data wrangling       
        "fBasics",     # Summary statistics 
        "corrplot",    # Correlations 
        "psych",       # Correlation p-values 
        "hdm",         # High-Dimensional Metrics
        "rpart",       # Classification and regression trees
        "rpart.plot",  # Plotting trees 
        "car",         # linear hypothesis testing for causal tree
        "aod")         # hypothesis testing 


**Non-CRAN packages:** The packages below have not yet been uploaded to CRAN or any other major package repository, but we can grab them from their authors' github webpages. If you don't have these packages installed already, uncomment the relevant line below to install it.

In [ ]:
# For causal trees (Athey and Imbens, 2016)  version 0.0
#library(devtools)    # Install packages from github 
#install_github("susanathey/causalTree")
require("causalTree")

## Loading the data

We will be working with the `welfare` dataset, from "Modeling Heterogeneous Treatment Effects in Survey Experiments with Baysian Additive Regression Trees" (Green and Kern, 2012) ).

Next, we load in the raw data and perform some data cleaning.

In [ ]:
# R script for reading data from github repository, set path to where you have the tutorial files saved.
df_experiment<-readRDS(url("https://github.com/ignaciomsarmiento/datasets/raw/main/welfare.rds"))


In [ ]:
head(df_experiment)

## Cleaning the data

Let's do some minimal housekeeping. First, we will drop the columns that aren't outcomes, treatments or (pre-treatment) covariates, since we won't be using those.


In [ ]:
covariate_names<- c("hrs1", "partyid", "income", "rincome", "wrkstat", "wrkslf", "age", "polviews", "educ", "earnrs", "race", "marital", "sibs", "childs", "occ80", "prestg80", "indus80", "res16", "reg16", "mobile16", "family16", "parborn", "maeduc", "degree", "sex", "born", "hompop", "babies", "preteen", "teens", "adults")

In [ ]:
# Combine all names
all_variables_names <- c("Y", "W", covariate_names)
df <- df_experiment %>% select(all_of(all_variables_names))

Next, let's drop any row that has missing values in them.

In [ ]:
# Drop rows containing missing values
df <- df %>% drop_na()

Some of the methods below don't accept `factor` variables, so let's change their type to `numeric` here. 

In [ ]:
# Converting all columns to numerical and add row id
df <- data.frame(lapply(df, function(x) as.numeric(as.character(x))))

df <- df %>% mutate_if(is.character,as.numeric)
df <- df %>% rowid_to_column( "ID")

head(df)

---

## Descriptive statistics

It's often useful to begin data analysis by simply looking at simple summary statistics. We use the function `basicStats` from the package `fBasics` to calculate them.

In [ ]:
# Make a data.frame containing summary statistics of interest
summ_stats <- fBasics::basicStats(df)
summ_stats <- as.data.frame(t(summ_stats))
# Rename some of the columns for convenience
summ_stats <- summ_stats %>% select("Mean", "Stdev", "Minimum", "1. Quartile", "Median",  "3. Quartile", "Maximum")
summ_stats <- summ_stats %>% rename('Lower quartile'= '1. Quartile', 'Upper quartile' ='3. Quartile')

In [ ]:
summ_stats

Presenting pairwise correlations is easy with the `corrplot` function from the `corrplot` package. On the table below, if the (unadjusted) p-value for a pair of variables is less than 0.05, its square is not colored.

In [ ]:
# Note: if the plot looks too cramped, try increasing fig.width and fig.height in the line above
pairwise_pvalues <- psych::corr.test(df, df)$p
corrplot(cor(df),
        type="upper",
        tl.col="black",
        order="hclust",
        tl.cex=1,
        addgrid.col = "black",
        p.mat=pairwise_pvalues,
        sig.level=0.05,
        number.font=10,
        insig="blank")

# PART I: ATE y CATE

 *"too much" money is spent on assistance to the Poor (treatment) or Welfare (control)*

In [ ]:
tapply(df$Y,df$W,mean)

- 48% agree that too much money is spent on Welfare
- 11% agree that too much money is spent on assistance to the Poor

In [ ]:
difference_in_means <- function(dataset) {
  # Filter treatment / control observations, pulls outcome variable as a vector
  y1 <- dataset %>% dplyr::filter(W == 1) %>% dplyr::pull(Y) # Outcome in treatment grp
  y0 <- dataset %>% dplyr::filter(W == 0) %>% dplyr::pull(Y) # Outcome in control group
  
  n1 <- sum(df[,"W"])     # Number of obs in treatment
  n0 <- sum(1 - df[,"W"]) # Number of obs in control
  
  # Difference in means is ATE
  tauhat <- mean(y1) - mean(y0)
  
  # 95% Confidence intervals
  se_hat <- sqrt( var(y0)/(n0-1) + var(y1)/(n1-1) )
  lower_ci <- tauhat - 1.96 * se_hat
  upper_ci <- tauhat + 1.96 * se_hat
  
  return(c(ATE = tauhat, lower_ci = lower_ci, upper_ci = upper_ci))
}

tauhat_rct <- difference_in_means(df)
tauhat_rct

In [ ]:
ols_ate= lm(Y~W, data=df)

In [ ]:
fmla= as.formula(paste("Y ~ W +", paste(covariate_names, collapse= "+")))
ols_cate= lm(fmla, data=df)
stargazer::stargazer(ols_ate,ols_cate,type="text")

# Part II: HTE, Causal Trees



-   Some authors consider the interaction between the treatment and attributions, e.g.

    -   Federico (2004) examines a complicated three-way interaction between the treatment, education, and racial perceptions.

    -   Jacoby (2000) suggests that party and ideology may make some respondents especially receptive to the more specific program (should strong and weak Democrats be treated as separate subgroups or should they be combined?)
        
Let's see how to analyze the data set selected above for heterogeneous treatment effects.        



-   We need to proceed in steps

-   **Step 1**: Split the dataset. Why? $\rightarrow$ Athey and Imbens
    innovation $\rightarrow$ **Honesty**
        
-   **Step 2**: Fit the tree
-   **Step 3**: Crossvalidate
-   **Step 4**: Predict point estimates (on estimation sample)
-   **Step 5**: Compute standard errors
-   **Step 6**: Predict point estimates (on test set)        
        


Before we start let’s separate a portion of our dataset as a test set. Later we will use this subset to evaluate the performance.

In [ ]:
train_fraction <- 0.80  # Use train_fraction % of the dataset to train our models

df_train <- sample_frac(df, replace=F, size=train_fraction)
df_test <- anti_join(df,df_train, by = "ID")#need to check on larger datasets

#### Step 1: Split the dataset

As we just explained, honesty requires us to separate different subsets of our training data for model selection and prediction.

+ `df_split`: the *splitting sample*, used to build the tree
+ `df_est`: the *estimation sample*, used to compute the average treatment effect in each leaf

In [ ]:
# Diving the data 40%-40%-20% into splitting, estimation and validation samples
split_size <- floor(nrow(df_train) * 0.5)
df_split <- sample_n(df_train, replace=FALSE, size=split_size)

# Make the splits
df_est <- anti_join(df_train,df_split, by ="ID")

#### Step 2: Fit the tree

Begin by defining a formula containing only the outcome and the covariates.

In [ ]:
fmla_ct <- paste("factor(Y) ~", paste(covariate_names, collapse = " + "))

print('This is our regression model')
print( fmla_ct)

Next, we use the `honest.causalTree` function from the `causalTree` package. To ensure that honesty is enabled, the parameters for splitting and cross-validation below should not be changed. However, if your tree is not splitting at all, try decreasing the parameter `minsize` that controls the minimum size of each leaf.

For more details on other parameters, please take a look at this extended [documentation](https://github.com/susanathey/causalTree/blob/master/briefintro.pdf) for the `causalTree` paper.


In [ ]:
ct_unpruned <- honest.causalTree(
  formula = fmla_ct,            # Define the model
  data = df_split,              # Subset used to create tree structure
  est_data = df_est,            # Which data set to use to estimate effects

  treatment = df_split$W,       # Splitting sample treatment variable
  est_treatment = df_est$W,     # Estimation sample treatment variable

  split.Rule = "CT",            # Define the splitting option
  cv.option = "TOT",            # Cross validation options
  cp = 0,                       # Complexity parameter

  split.Honest = TRUE,          # Use honesty when splitting
  cv.Honest = TRUE,             # Use honesty when performing cross-validation

  minsize = 10,                 # Min. number of treatment and control cases in each leaf
  HonestSampleSize = nrow(df_est)) # Num obs used in estimation after building the tree

The resulting object will be `rpart` objects, so `rpart` methods for cross-validation and plotting can be used.

#### Step 3: Cross-validate

We must prune the tree by cross-validation to avoid overfitting. The honest cross-validation method selected above (and recommended) penalizes an estimate of the variance in the treatment effects estimates across leaves, and this estimate is computed using the estimation sample. The `cv.option` selected above ($TOT$) uses an unbiased estimate of the test mean-squared error.


In [ ]:
# Table of cross-validated values by tuning parameter.
ct_cptable <- as.data.frame(ct_unpruned$cptable)

# Obtain optimal complexity parameter to prune tree.
selected_cp <- which.min(ct_cptable$xerror)
optim_cp_ct <- ct_cptable[selected_cp, "CP"]

# Prune the tree at optimal complexity parameter.
ct_pruned <- prune(tree = ct_unpruned, cp = optim_cp_ct)

In [ ]:
rpart.plot(ct_pruned)

#### Step 4: Predict point estimates (on estimation sample)

To predict the treatment effect on the estimation sample, use the function `predict` as below.

In [ ]:
tauhat_ct_est <- predict(ct_pruned, newdata = df_est)


#### Step 5: Compute standard errors

The `causalTree` package does not compute standard errors by default, but we can compute them using the following trick. First, define $L_{\ell}$ to indicate assignment to leaf $\ell$ and consider the following linear model.

\begin{align}
Y = \sum_{\ell} L_{\ell}\alpha_{\ell} + W \cdot L_{\ell} \beta_{\ell}
\end{align}

In [ ]:
# Create a factor column 'leaf' indicating leaf assignment
num_leaves <- length(unique(tauhat_ct_est))  #There are as many leaves as there are predictions

df_est$leaf <- factor(tauhat_ct_est, labels = seq(num_leaves))

# Run the regression
ols_ct <- lm(as.formula("Y ~ 0 + leaf + W:leaf"), data= df_est) #Warning: the tree won't split for charitable dataset
print(as.formula("Y ~ 0 + leaf + W:leaf"))

The interaction coefficients in this regression recover the average treatment effects in each leaf, since

\begin{align}
  E[Y|W=1, L=1] - E[Y|W=0, L=1] = (\alpha_{1} + \beta_{1}) - (\alpha_{1}) = \beta_{1}
\end{align}

Therefore, the standard error around the coefficients is also the standard error around the treatment effects. In the next subsection, we will also use these statistics to test hypothesis about leaf estimates.

In [ ]:
ols_ct_summary <- summary(ols_ct)
te_summary <- coef(ols_ct_summary)[(num_leaves+1):(2*num_leaves), c("Estimate", "Std. Error")]
te_summary

#### Step 6: Predict point estimates (on test set)

To predict the treatment effect on a new, entirely unseen data, use the function `predict` as below.

In [ ]:
tauhat_ct_test <- predict(ct_pruned, newdata = df_test)

### Assessing heterogeneity

A natural place to begin is by ploting the (pruned) tree. We can use the `rpart.plot` function from the `rpart.plot` package.

In [ ]:
rpart.plot(
  x = ct_pruned,        # Pruned tree
  type = 3,             # Draw separate split labels for the left and right directions
  fallen = TRUE,        # Position the leaf nodes at the bottom of the graph
  leaf.round = 1,       # Rounding of the corners of the leaf node boxes
  extra = 100,          # Display the percentage of observations in the node
  branch = 0.1,          # Shape of the branch lines
  box.palette = "RdBu") # Palette for coloring the node

#### Treatment effect heterogeneity

We may want to test the treatment effect is different across leaves. That is, to test the null hypothesis that

$$
 E[Y|W=1, L=1] - E[Y|W=0, L=1] =  E[Y|W=1, L=\ell] - E[Y|W=0, L=\ell] \quad \text{for all } \ell > 1
$$

Following the linear model setup described in Step 5 in the previous subsection, we can use the function `linearHypothesis` from the `car` package to test this hypothesis.

In [ ]:
# Null hypothesis: all leaf values are the same
hypothesis <- paste0("leaf1:W = leaf", seq(2, num_leaves), ":W")
ftest <- linearHypothesis(ols_ct, hypothesis, test="F")
ftest

Next, we test if the average treatment effect is different between all two pairs of leaves. Note that here we are not performing any type of multiple hypothesis testing correction.

In [ ]:
# Null hypothesis: leaf i = leaf k for all i != k
p_values_leaf_by_leaf <- matrix(nrow = num_leaves, ncol = num_leaves)
differences_leaf_by_leaf <- matrix(nrow = num_leaves, ncol = num_leaves)
stderror_leaf_by_leaf <- matrix(nrow = num_leaves, ncol = num_leaves)
hypotheses_grid <- combn(1:num_leaves, 2)
summ <- coef(summary(ols_ct))

invisible(apply(hypotheses_grid, 2, function(x) {
  leafi <- paste0("leaf", x[1], ":W")
  leafj <- paste0("leaf", x[2], ":W")
  hypothesis <- paste0(leafi, " = ", leafj)

  differences_leaf_by_leaf[x[2], x[1]] <<- summ[leafj, 1] - summ[leafi, 1]
  stderror_leaf_by_leaf[x[2], x[1]] <<- sqrt(summ[leafj, 2]^2 + summ[leafi, 2]^2)
  p_values_leaf_by_leaf[x[2], x[1]] <<- linearHypothesis(ols_ct, hypothesis)[2, "Pr(>F)"]
}))


<caption>
Pairwise leaf differences:<br> Average treatment effect differences between leaf i and leaf j
</caption>
<thead>
<tr>
<th style="text-align:left;">
rowname
</th>
<th style="text-align:left;">
leaf1
</th>
<th style="text-align:left;">
leaf2
</th>
<th style="text-align:left;">
leaf3
</th>
<th style="text-align:left;">
leaf4
</th>
<th style="text-align:left;">
leaf5
</th>
<th style="text-align:left;">
leaf6
</th>
<th style="text-align:left;">
leaf7
</th>
<th style="text-align:left;">
leaf8
</th>
<th style="text-align:left;">
leaf9
</th>
<th style="text-align:left;">
leaf10
</th>
<th style="text-align:left;">
leaf11
</th>
<th style="text-align:left;">
leaf12
</th>
<th style="text-align:left;">
leaf13
</th>
<th style="text-align:left;">
leaf14
</th>
<th style="text-align:left;">
leaf15
</th>
<th style="text-align:left;">
leaf16
</th>
<th style="text-align:left;">
leaf17
</th>
<th style="text-align:left;">
leaf18
</th>
<th style="text-align:left;">
leaf19
</th>
<th style="text-align:left;">
leaf20
</th>
<th style="text-align:left;">
leaf21
</th>
<th style="text-align:left;">
leaf22
</th>
<th style="text-align:left;">
leaf23
</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf1</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf2</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.002</span> <br> (0.15)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf3</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.109</span> <br> (0.151)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.106</span> <br> (0.114)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf4</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.126</span> <br> (0.13)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.124</span> <br> (0.084)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.018</span> <br> (0.086)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf5</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.141</span> <br> (0.13)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.139</span> <br> (0.085)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.033</span> <br> (0.086)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.015</span> <br> (0.039)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf6</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.165</span> <br> (0.171)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.163</span> <br> (0.14)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.056</span> <br> (0.141)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.039</span> <br> (0.118)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.024</span> <br> (0.118)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf7</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.178</span> <br> (0.211)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.176</span> <br> (0.186)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.069</span> <br> (0.186)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.052</span> <br> (0.17)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.037</span> <br> (0.17)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.013</span> <br> (0.203)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf8</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.182</span> <br> (0.147)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.18</span> <br> (0.108)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.073</span> <br> (0.109)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.056</span> <br> (0.078)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.04</span> <br> (0.078)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.017</span> <br> (0.136)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.004</span> <br> (0.183)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf9</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.193</span> <br> (0.13)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.191</span> <br> (0.083)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.084</span> <br> (0.085)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.067</span> <br> (0.036)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.052</span> <br> (0.037)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.028</span> <br> (0.117)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.015</span> <br> (0.169)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.011</span> <br> (0.077)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf10</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.194</span> <br> (0.157)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.192</span> <br> (0.122)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.085</span> <br> (0.123)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.068</span> <br> (0.096)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.053</span> <br> (0.096)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.029</span> <br> (0.147)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.016</span> <br> (0.191)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.012</span> <br> (0.117)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.001</span> <br> (0.095)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf11</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.217</span> <br> (0.162)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.215</span> <br> (0.128)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.109</span> <br> (0.129)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.091</span> <br> (0.104)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.076</span> <br> (0.104)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.052</span> <br> (0.152)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.039</span> <br> (0.195)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.036</span> <br> (0.124)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.024</span> <br> (0.103)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.024</span> <br> (0.136)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf12</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.223</span> <br> (0.139)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.22</span> <br> (0.097)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.114</span> <br> (0.099)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.096</span> <br> (0.062)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.081</span> <br> (0.062)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.058</span> <br> (0.127)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.045</span> <br> (0.177)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.041</span> <br> (0.092)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.03</span> <br> (0.061)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.029</span> <br> (0.107)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.005</span> <br> (0.115)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf13</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.238</span> <br> (0.142)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.236</span> <br> (0.102)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.13</span> <br> (0.103)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.112</span> <br> (0.069)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.097</span> <br> (0.069)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.073</span> <br> (0.131)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.06</span> <br> (0.179)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.056</span> <br> (0.096)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.045</span> <br> (0.068)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.044</span> <br> (0.111)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.021</span> <br> (0.119)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.016</span> <br> (0.084)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf14</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.239</span> <br> (0.141)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.237</span> <br> (0.1)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.13</span> <br> (0.101)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.113</span> <br> (0.066)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.097</span> <br> (0.066)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.074</span> <br> (0.129)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.061</span> <br> (0.178)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.057</span> <br> (0.094)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.046</span> <br> (0.065)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.045</span> <br> (0.11)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.021</span> <br> (0.117)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.016</span> <br> (0.082)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.001</span> <br> (0.087)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf15</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.247</span> <br> (0.201)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.244</span> <br> (0.174)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.138</span> <br> (0.175)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.12</span> <br> (0.157)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.105</span> <br> (0.157)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.082</span> <br> (0.193)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.069</span> <br> (0.228)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.065</span> <br> (0.171)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.054</span> <br> (0.157)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.053</span> <br> (0.18)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.029</span> <br> (0.185)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.024</span> <br> (0.165)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.008</span> <br> (0.167)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.008</span> <br> (0.166)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf16</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.259</span> <br> (0.135)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.257</span> <br> (0.091)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.15</span> <br> (0.093)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.133</span> <br> (0.052)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.117</span> <br> (0.052)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.094</span> <br> (0.123)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.081</span> <br> (0.173)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.077</span> <br> (0.085)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.066</span> <br> (0.051)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.065</span> <br> (0.102)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.041</span> <br> (0.11)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.036</span> <br> (0.071)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.021</span> <br> (0.077)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.02</span> <br> (0.075)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.012</span> <br> (0.161)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf17</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.26</span> <br> (0.165)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.258</span> <br> (0.132)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.152</span> <br> (0.133)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.134</span> <br> (0.108)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.119</span> <br> (0.108)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.095</span> <br> (0.155)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.082</span> <br> (0.198)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.079</span> <br> (0.128)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.067</span> <br> (0.108)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.067</span> <br> (0.139)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.043</span> <br> (0.145)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.038</span> <br> (0.119)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.022</span> <br> (0.122)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.022</span> <br> (0.121)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.014</span> <br> (0.187)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.002</span> <br> (0.114)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf18</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.281</span> <br> (0.153)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.278</span> <br> (0.116)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.172</span> <br> (0.117)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.154</span> <br> (0.089)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.139</span> <br> (0.089)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.116</span> <br> (0.142)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.103</span> <br> (0.188)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.099</span> <br> (0.112)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.088</span> <br> (0.088)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.087</span> <br> (0.125)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.063</span> <br> (0.131)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.058</span> <br> (0.101)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.042</span> <br> (0.106)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.042</span> <br> (0.104)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.034</span> <br> (0.177)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.022</span> <br> (0.096)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.02</span> <br> (0.135)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf19</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.287</span> <br> (0.141)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.284</span> <br> (0.101)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.178</span> <br> (0.102)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.16</span> <br> (0.067)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.145</span> <br> (0.067)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.122</span> <br> (0.13)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.109</span> <br> (0.178)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.105</span> <br> (0.095)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.094</span> <br> (0.066)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.093</span> <br> (0.11)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.069</span> <br> (0.118)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.064</span> <br> (0.083)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.048</span> <br> (0.088)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.048</span> <br> (0.086)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.04</span> <br> (0.167)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.028</span> <br> (0.076)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.026</span> <br> (0.121)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.006</span> <br> (0.104)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf20</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.32</span> <br> (0.232)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.317</span> <br> (0.21)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.211</span> <br> (0.211)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.194</span> <br> (0.196)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.178</span> <br> (0.196)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.155</span> <br> (0.225)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.142</span> <br> (0.256)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.138</span> <br> (0.207)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.127</span> <br> (0.196)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.126</span> <br> (0.215)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.102</span> <br> (0.219)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.097</span> <br> (0.202)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.082</span> <br> (0.204)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.081</span> <br> (0.203)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.073</span> <br> (0.248)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.061</span> <br> (0.199)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.059</span> <br> (0.221)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.039</span> <br> (0.212)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.033</span> <br> (0.204)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf21</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.357</span> <br> (0.137)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.355</span> <br> (0.094)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.249</span> <br> (0.095)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.231</span> <br> (0.056)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.216</span> <br> (0.057)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.192</span> <br> (0.125)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.179</span> <br> (0.175)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.176</span> <br> (0.088)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.164</span> <br> (0.055)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.163</span> <br> (0.104)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.14</span> <br> (0.112)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.135</span> <br> (0.074)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.119</span> <br> (0.08)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.119</span> <br> (0.078)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.111</span> <br> (0.163)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.098</span> <br> (0.067)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.097</span> <br> (0.116)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.077</span> <br> (0.098)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.071</span> <br> (0.079)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.038</span> <br> (0.2)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf22</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.407</span> <br> (0.148)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.405</span> <br> (0.11)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.298</span> <br> (0.111)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.281</span> <br> (0.08)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.266</span> <br> (0.08)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.242</span> <br> (0.137)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.229</span> <br> (0.184)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.225</span> <br> (0.105)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.214</span> <br> (0.079)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.213</span> <br> (0.119)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.19</span> <br> (0.126)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.184</span> <br> (0.094)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.169</span> <br> (0.099)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.168</span> <br> (0.097)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.16</span> <br> (0.173)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.148</span> <br> (0.088)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.147</span> <br> (0.129)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.126</span> <br> (0.113)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.12</span> <br> (0.097)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.087</span> <br> (0.208)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.05</span> <br> (0.09)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style="     color: gray !important;">leaf23</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.413</span> <br> (0.239)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: white !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray !important;">0.41</span> <br> (0.217)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.304</span> <br> (0.218)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.286</span> <br> (0.203)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.271</span> <br> (0.204)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.247</span> <br> (0.232)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.234</span> <br> (0.262)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.231</span> <br> (0.214)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.22</span> <br> (0.203)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.219</span> <br> (0.222)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.195</span> <br> (0.225)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.19</span> <br> (0.209)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.174</span> <br> (0.211)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.174</span> <br> (0.21)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.166</span> <br> (0.254)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.154</span> <br> (0.207)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.152</span> <br> (0.227)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.132</span> <br> (0.219)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.126</span> <br> (0.211)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.093</span> <br> (0.28)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.055</span> <br> (0.208)</span>
</td>
<td style="text-align:left;">
<span style="     color: gray !important;"><span style="     color: gray !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: white !important;">0.005</span> <br> (0.215)</span>
</td>
<td style="text-align:left;">
<span style="     color: white !important;">NA</span>
</td>
</tr>
</tbody>
<tfoot>
<tr>
<td style="padding: 0; " colspan="100%">
<span style="font-style: italic;">Note: </span>
</td>
</tr>
<tr>
<td style="padding: 0; " colspan="100%">
<sup></sup> Standard errors in parenthesis. Significance (not adjusted for multiple testing):
<ul>
<li>
No background color: p ≥ 0.1
</li>
<li>
<span style="color: white;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: gray;">Gray</span> background: p &lt; 0.1
</li>
<li>
<span style="color: white;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: black;">Black</span> background: p &lt; 0.05
</li>
</ul>
</td>
</tr>
</tfoot>


#### Covariate heterogeneity

Another measure of heterogeneity is how much the average level of each covariate changes across leaves. 


In [ ]:
# Null hypothesis: the mean is equal across all leaves
hypothesis <- paste0("leaf1 = leaf", seq(2, num_leaves))
means_per_leaf <- matrix(nrow = num_leaves, ncol = num_leaves)
significance <- matrix(nrow = 2, ncol=length(covariate_names))

# Regress each covariate on leaf assignment to means p
cov_means <- lapply(covariate_names, function(covariate) {
  lm(paste0(covariate, ' ~ 0 + leaf'), data = df_est)
})

# Extract the mean and standard deviation of each covariate per leaf
cov_table <- lapply(cov_means, function(cov_mean) {
  as.data.frame(t(coef(summary(cov_mean))[,c("Estimate", "Std. Error")]))
})

# Test if means are the same across leaves
cov_ftests <- sapply(cov_means, function(cov_mean) {
  # Sometimes the regression has no residual (SSE = 0), 
  # so we cannot perform an F-test
  tryCatch({
    linearHypothesis(cov_mean, hypothesis)[2, c("F", "Pr(>F)")]
  },
    error = function(cond) {
      message(paste0("Error message during F-test for`", cov_mean$terms[[2]], "`:"))
      message(cond)
      return(c("F" = NA, "Pr(>F)" = NA))
    })
})


<caption>
Average covariate values in each leaf
</caption>
<thead>
<tr>
<th style="text-align:left;">
covariates
</th>
<th style="text-align:left;">
leaf1
</th>
<th style="text-align:left;">
leaf2
</th>
<th style="text-align:left;">
leaf3
</th>
<th style="text-align:left;">
leaf4
</th>
<th style="text-align:left;">
leaf5
</th>
<th style="text-align:left;">
leaf6
</th>
<th style="text-align:left;">
leaf7
</th>
<th style="text-align:left;">
leaf8
</th>
<th style="text-align:left;">
leaf9
</th>
<th style="text-align:left;">
leaf10
</th>
<th style="text-align:left;">
leaf11
</th>
<th style="text-align:left;">
leaf12
</th>
<th style="text-align:left;">
leaf13
</th>
<th style="text-align:left;">
leaf14
</th>
<th style="text-align:left;">
leaf15
</th>
<th style="text-align:left;">
leaf16
</th>
<th style="text-align:left;">
leaf17
</th>
<th style="text-align:left;">
leaf18
</th>
<th style="text-align:left;">
leaf19
</th>
<th style="text-align:left;">
leaf20
</th>
<th style="text-align:left;">
leaf21
</th>
<th style="text-align:left;">
leaf22
</th>
<th style="text-align:left;">
leaf23
</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">hrs1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 126, 142, 1) !important;">34.75</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">41.61</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(49, 181, 123, 1) !important;">57.82</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 125, 142, 1) !important;">34.39</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 173, 129, 1) !important;">54.62</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">39</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 129, 142, 1) !important;">35.83</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 125, 142, 1) !important;">34.29</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">41.77</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(57, 186, 118, 1) !important;">59.9</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">44.79</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">40.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">41.95</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">44.27</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">37.82</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(41, 123, 142, 1) !important;">33.52</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">45.16</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">43.62</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">37.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">47.44</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">42.24</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">36.27</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(41, 122, 142, 1) !important;">33</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(1.718)
</td>
<td style="text-align:left;">
(1.14)
</td>
<td style="text-align:left;">
(1.184)
</td>
<td style="text-align:left;">
(0.392)
</td>
<td style="text-align:left;">
(0.401)
</td>
<td style="text-align:left;">
(1.666)
</td>
<td style="text-align:left;">
(2.429)
</td>
<td style="text-align:left;">
(1.048)
</td>
<td style="text-align:left;">
(0.348)
</td>
<td style="text-align:left;">
(1.322)
</td>
<td style="text-align:left;">
(1.454)
</td>
<td style="text-align:left;">
(0.808)
</td>
<td style="text-align:left;">
(0.918)
</td>
<td style="text-align:left;">
(0.873)
</td>
<td style="text-align:left;">
(2.249)
</td>
<td style="text-align:left;">
(0.648)
</td>
<td style="text-align:left;">
(1.524)
</td>
<td style="text-align:left;">
(1.227)
</td>
<td style="text-align:left;">
(0.889)
</td>
<td style="text-align:left;">
(2.805)
</td>
<td style="text-align:left;">
(0.72)
</td>
<td style="text-align:left;">
(1.095)
</td>
<td style="text-align:left;">
(2.886)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">partyid</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 166, 133, 1) !important;">4.271</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(48, 106, 142, 1) !important;">0.651</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 171, 130, 1) !important;">4.525</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 168, 132, 1) !important;">4.379</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 166, 133, 1) !important;">4.296</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(47, 107, 142, 1) !important;">0.686</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 136, 1) !important;">3.917</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 162, 135, 1) !important;">4.047</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(48, 105, 142, 1) !important;">0.612</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 172, 130, 1) !important;">4.593</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(48, 106, 142, 1) !important;">0.672</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">3.364</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 160, 136, 1) !important;">3.851</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">3.613</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">2.964</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">3.988</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(48, 104, 142, 1) !important;">0.557</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">3.266</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">3.698</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(47, 107, 142, 1) !important;">0.722</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(49, 103, 142, 1) !important;">0.491</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 164, 134, 1) !important;">4.144</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 158, 137, 1) !important;">3.765</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.17)
</td>
<td style="text-align:left;">
(0.113)
</td>
<td style="text-align:left;">
(0.117)
</td>
<td style="text-align:left;">
(0.039)
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.165)
</td>
<td style="text-align:left;">
(0.241)
</td>
<td style="text-align:left;">
(0.104)
</td>
<td style="text-align:left;">
(0.034)
</td>
<td style="text-align:left;">
(0.131)
</td>
<td style="text-align:left;">
(0.144)
</td>
<td style="text-align:left;">
(0.08)
</td>
<td style="text-align:left;">
(0.091)
</td>
<td style="text-align:left;">
(0.086)
</td>
<td style="text-align:left;">
(0.223)
</td>
<td style="text-align:left;">
(0.064)
</td>
<td style="text-align:left;">
(0.151)
</td>
<td style="text-align:left;">
(0.122)
</td>
<td style="text-align:left;">
(0.088)
</td>
<td style="text-align:left;">
(0.278)
</td>
<td style="text-align:left;">
(0.071)
</td>
<td style="text-align:left;">
(0.108)
</td>
<td style="text-align:left;">
(0.286)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">income</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">11.29</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">12</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">11.61</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">11.62</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">11.6</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(47, 108, 142, 1) !important;">9.529</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">11.25</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">11.03</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">11.32</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">11.64</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">11.42</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">11.41</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">11.54</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">11.2</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">11.46</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(43, 117, 142, 1) !important;">10.01</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">11.43</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">11.81</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(41, 123, 142, 1) !important;">10.26</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">11.11</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">11.38</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">11.08</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(52, 96, 141, 1) !important;">9</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.221)
</td>
<td style="text-align:left;">
(0.146)
</td>
<td style="text-align:left;">
(0.152)
</td>
<td style="text-align:left;">
(0.05)
</td>
<td style="text-align:left;">
(0.052)
</td>
<td style="text-align:left;">
(0.214)
</td>
<td style="text-align:left;">
(0.312)
</td>
<td style="text-align:left;">
(0.135)
</td>
<td style="text-align:left;">
(0.045)
</td>
<td style="text-align:left;">
(0.17)
</td>
<td style="text-align:left;">
(0.187)
</td>
<td style="text-align:left;">
(0.104)
</td>
<td style="text-align:left;">
(0.118)
</td>
<td style="text-align:left;">
(0.112)
</td>
<td style="text-align:left;">
(0.289)
</td>
<td style="text-align:left;">
(0.083)
</td>
<td style="text-align:left;">
(0.196)
</td>
<td style="text-align:left;">
(0.158)
</td>
<td style="text-align:left;">
(0.114)
</td>
<td style="text-align:left;">
(0.36)
</td>
<td style="text-align:left;">
(0.093)
</td>
<td style="text-align:left;">
(0.141)
</td>
<td style="text-align:left;">
(0.371)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">rincome</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">10.71</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">10.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">10.99</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">9.773</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 156, 137, 1) !important;">11.17</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 120, 142, 1) !important;">8.235</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">11</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">10.33</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">10.25</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">11.09</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">10.18</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">10.54</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">10.61</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">10.36</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">10.5</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 125, 142, 1) !important;">8.662</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">10.85</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">11.4</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 118, 142, 1) !important;">8.128</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">10.06</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">10.55</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">9.847</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(43, 116, 142, 1) !important;">8</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.377)
</td>
<td style="text-align:left;">
(0.25)
</td>
<td style="text-align:left;">
(0.26)
</td>
<td style="text-align:left;">
(0.086)
</td>
<td style="text-align:left;">
(0.088)
</td>
<td style="text-align:left;">
(0.366)
</td>
<td style="text-align:left;">
(0.533)
</td>
<td style="text-align:left;">
(0.23)
</td>
<td style="text-align:left;">
(0.076)
</td>
<td style="text-align:left;">
(0.29)
</td>
<td style="text-align:left;">
(0.319)
</td>
<td style="text-align:left;">
(0.177)
</td>
<td style="text-align:left;">
(0.201)
</td>
<td style="text-align:left;">
(0.191)
</td>
<td style="text-align:left;">
(0.493)
</td>
<td style="text-align:left;">
(0.142)
</td>
<td style="text-align:left;">
(0.334)
</td>
<td style="text-align:left;">
(0.269)
</td>
<td style="text-align:left;">
(0.195)
</td>
<td style="text-align:left;">
(0.615)
</td>
<td style="text-align:left;">
(0.158)
</td>
<td style="text-align:left;">
(0.24)
</td>
<td style="text-align:left;">
(0.633)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">wrkstat</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.146</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">1.092</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">1.03</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.229</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">1.011</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.216</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.208</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">1.194</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.16</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">1.012</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.06</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.207</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.125</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.075</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">1.286</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">1.315</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.148</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.138</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">1.313</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.167</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.179</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.169</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">1.235</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.051)
</td>
<td style="text-align:left;">
(0.034)
</td>
<td style="text-align:left;">
(0.035)
</td>
<td style="text-align:left;">
(0.012)
</td>
<td style="text-align:left;">
(0.012)
</td>
<td style="text-align:left;">
(0.049)
</td>
<td style="text-align:left;">
(0.072)
</td>
<td style="text-align:left;">
(0.031)
</td>
<td style="text-align:left;">
(0.01)
</td>
<td style="text-align:left;">
(0.039)
</td>
<td style="text-align:left;">
(0.043)
</td>
<td style="text-align:left;">
(0.024)
</td>
<td style="text-align:left;">
(0.027)
</td>
<td style="text-align:left;">
(0.026)
</td>
<td style="text-align:left;">
(0.066)
</td>
<td style="text-align:left;">
(0.019)
</td>
<td style="text-align:left;">
(0.045)
</td>
<td style="text-align:left;">
(0.036)
</td>
<td style="text-align:left;">
(0.026)
</td>
<td style="text-align:left;">
(0.083)
</td>
<td style="text-align:left;">
(0.021)
</td>
<td style="text-align:left;">
(0.032)
</td>
<td style="text-align:left;">
(0.085)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">wrkslf</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.854</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.899</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(71, 44, 122, 1) !important;">1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">1.871</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">2</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.941</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">1.917</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.891</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.921</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(71, 44, 122, 1) !important;">1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">1.806</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.857</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.863</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.866</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.893</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.923</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 125, 142, 1) !important;">1.689</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.862</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.922</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">1.944</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.879</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.941</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">2</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.026)
</td>
<td style="text-align:left;">
(0.027)
</td>
<td style="text-align:left;">
(0.009)
</td>
<td style="text-align:left;">
(0.009)
</td>
<td style="text-align:left;">
(0.039)
</td>
<td style="text-align:left;">
(0.056)
</td>
<td style="text-align:left;">
(0.024)
</td>
<td style="text-align:left;">
(0.008)
</td>
<td style="text-align:left;">
(0.031)
</td>
<td style="text-align:left;">
(0.034)
</td>
<td style="text-align:left;">
(0.019)
</td>
<td style="text-align:left;">
(0.021)
</td>
<td style="text-align:left;">
(0.02)
</td>
<td style="text-align:left;">
(0.052)
</td>
<td style="text-align:left;">
(0.015)
</td>
<td style="text-align:left;">
(0.035)
</td>
<td style="text-align:left;">
(0.028)
</td>
<td style="text-align:left;">
(0.021)
</td>
<td style="text-align:left;">
(0.065)
</td>
<td style="text-align:left;">
(0.017)
</td>
<td style="text-align:left;">
(0.025)
</td>
<td style="text-align:left;">
(0.067)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">age</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(51, 182, 121, 1) !important;">54.44</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 136, 1) !important;">46.43</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">45.28</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">41.86</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">39.45</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">44.02</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(46, 179, 124, 1) !important;">53</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">42.45</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">41.76</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 154, 138, 1) !important;">44.26</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 120, 142, 1) !important;">32</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">38.55</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">41.98</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">39.07</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">38.07</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">37.53</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 173, 130, 1) !important;">50.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">40.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 129, 142, 1) !important;">34.84</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 119, 142, 1) !important;">31.44</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">38.11</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(44, 115, 142, 1) !important;">30.1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 120, 142, 1) !important;">31.76</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(1.698)
</td>
<td style="text-align:left;">
(1.127)
</td>
<td style="text-align:left;">
(1.17)
</td>
<td style="text-align:left;">
(0.387)
</td>
<td style="text-align:left;">
(0.396)
</td>
<td style="text-align:left;">
(1.647)
</td>
<td style="text-align:left;">
(2.401)
</td>
<td style="text-align:left;">
(1.036)
</td>
<td style="text-align:left;">
(0.344)
</td>
<td style="text-align:left;">
(1.307)
</td>
<td style="text-align:left;">
(1.437)
</td>
<td style="text-align:left;">
(0.798)
</td>
<td style="text-align:left;">
(0.907)
</td>
<td style="text-align:left;">
(0.863)
</td>
<td style="text-align:left;">
(2.223)
</td>
<td style="text-align:left;">
(0.641)
</td>
<td style="text-align:left;">
(1.506)
</td>
<td style="text-align:left;">
(1.213)
</td>
<td style="text-align:left;">
(0.879)
</td>
<td style="text-align:left;">
(2.772)
</td>
<td style="text-align:left;">
(0.712)
</td>
<td style="text-align:left;">
(1.083)
</td>
<td style="text-align:left;">
(2.853)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">polviews</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 163, 134, 1) !important;">4.857</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">3.894</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 167, 133, 1) !important;">4.993</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 163, 134, 1) !important;">4.857</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 164, 134, 1) !important;">4.887</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">4.22</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 160, 136, 1) !important;">4.708</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 136, 1) !important;">4.727</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">3.567</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 166, 133, 1) !important;">4.989</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">3.853</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(49, 103, 142, 1) !important;">2.442</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 160, 136, 1) !important;">4.701</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(49, 102, 142, 1) !important;">2.398</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(51, 99, 141, 1) !important;">2.286</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">4.767</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">3.711</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(49, 103, 142, 1) !important;">2.436</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(49, 103, 142, 1) !important;">2.43</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">4.444</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 118, 142, 1) !important;">3.026</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">4.793</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 158, 137, 1) !important;">4.647</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.148)
</td>
<td style="text-align:left;">
(0.098)
</td>
<td style="text-align:left;">
(0.102)
</td>
<td style="text-align:left;">
(0.034)
</td>
<td style="text-align:left;">
(0.035)
</td>
<td style="text-align:left;">
(0.144)
</td>
<td style="text-align:left;">
(0.209)
</td>
<td style="text-align:left;">
(0.09)
</td>
<td style="text-align:left;">
(0.03)
</td>
<td style="text-align:left;">
(0.114)
</td>
<td style="text-align:left;">
(0.125)
</td>
<td style="text-align:left;">
(0.07)
</td>
<td style="text-align:left;">
(0.079)
</td>
<td style="text-align:left;">
(0.075)
</td>
<td style="text-align:left;">
(0.194)
</td>
<td style="text-align:left;">
(0.056)
</td>
<td style="text-align:left;">
(0.131)
</td>
<td style="text-align:left;">
(0.106)
</td>
<td style="text-align:left;">
(0.077)
</td>
<td style="text-align:left;">
(0.242)
</td>
<td style="text-align:left;">
(0.062)
</td>
<td style="text-align:left;">
(0.094)
</td>
<td style="text-align:left;">
(0.249)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">educ</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">15.4</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">13.34</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">14.91</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">13.76</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">14.39</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(44, 115, 142, 1) !important;">11.63</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">13.88</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">13.57</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">14.05</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">13.53</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 128, 142, 1) !important;">12.61</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 163, 134, 1) !important;">15.57</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">13.53</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">13.35</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">13.61</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 127, 142, 1) !important;">12.57</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 119, 142, 1) !important;">11.92</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 166, 133, 1) !important;">15.82</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">12.91</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 128, 142, 1) !important;">12.67</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 166, 133, 1) !important;">15.79</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 154, 138, 1) !important;">14.83</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">12.82</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.372)
</td>
<td style="text-align:left;">
(0.247)
</td>
<td style="text-align:left;">
(0.257)
</td>
<td style="text-align:left;">
(0.085)
</td>
<td style="text-align:left;">
(0.087)
</td>
<td style="text-align:left;">
(0.361)
</td>
<td style="text-align:left;">
(0.527)
</td>
<td style="text-align:left;">
(0.227)
</td>
<td style="text-align:left;">
(0.075)
</td>
<td style="text-align:left;">
(0.287)
</td>
<td style="text-align:left;">
(0.315)
</td>
<td style="text-align:left;">
(0.175)
</td>
<td style="text-align:left;">
(0.199)
</td>
<td style="text-align:left;">
(0.189)
</td>
<td style="text-align:left;">
(0.488)
</td>
<td style="text-align:left;">
(0.141)
</td>
<td style="text-align:left;">
(0.33)
</td>
<td style="text-align:left;">
(0.266)
</td>
<td style="text-align:left;">
(0.193)
</td>
<td style="text-align:left;">
(0.608)
</td>
<td style="text-align:left;">
(0.156)
</td>
<td style="text-align:left;">
(0.238)
</td>
<td style="text-align:left;">
(0.626)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">earnrs</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 123, 142, 1) !important;">1.229</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 156, 137, 1) !important;">2.037</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">1.822</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">2.1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.711</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 131, 142, 1) !important;">1.431</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(44, 115, 142, 1) !important;">1.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 128, 142, 1) !important;">1.349</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.715</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">1.852</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.776</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.654</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">1.827</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.753</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">1.929</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 129, 142, 1) !important;">1.374</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">1.852</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.745</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">1.726</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.722</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.637</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">1.449</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">1.471</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.115)
</td>
<td style="text-align:left;">
(0.076)
</td>
<td style="text-align:left;">
(0.079)
</td>
<td style="text-align:left;">
(0.026)
</td>
<td style="text-align:left;">
(0.027)
</td>
<td style="text-align:left;">
(0.111)
</td>
<td style="text-align:left;">
(0.162)
</td>
<td style="text-align:left;">
(0.07)
</td>
<td style="text-align:left;">
(0.023)
</td>
<td style="text-align:left;">
(0.088)
</td>
<td style="text-align:left;">
(0.097)
</td>
<td style="text-align:left;">
(0.054)
</td>
<td style="text-align:left;">
(0.061)
</td>
<td style="text-align:left;">
(0.058)
</td>
<td style="text-align:left;">
(0.15)
</td>
<td style="text-align:left;">
(0.043)
</td>
<td style="text-align:left;">
(0.102)
</td>
<td style="text-align:left;">
(0.082)
</td>
<td style="text-align:left;">
(0.059)
</td>
<td style="text-align:left;">
(0.187)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.073)
</td>
<td style="text-align:left;">
(0.193)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">race</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">1.062</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 171, 130, 1) !important;">1.697</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.089</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.095</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.11</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 156, 137, 1) !important;">1.451</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.167</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.109</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">1.368</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.123</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">1.463</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.221</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(51, 182, 121, 1) !important;">1.899</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.204</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.179</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.184</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.344</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.191</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.279</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 173, 130, 1) !important;">1.722</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.293</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.169</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.118</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.077)
</td>
<td style="text-align:left;">
(0.051)
</td>
<td style="text-align:left;">
(0.053)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.074)
</td>
<td style="text-align:left;">
(0.108)
</td>
<td style="text-align:left;">
(0.047)
</td>
<td style="text-align:left;">
(0.016)
</td>
<td style="text-align:left;">
(0.059)
</td>
<td style="text-align:left;">
(0.065)
</td>
<td style="text-align:left;">
(0.036)
</td>
<td style="text-align:left;">
(0.041)
</td>
<td style="text-align:left;">
(0.039)
</td>
<td style="text-align:left;">
(0.1)
</td>
<td style="text-align:left;">
(0.029)
</td>
<td style="text-align:left;">
(0.068)
</td>
<td style="text-align:left;">
(0.055)
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.125)
</td>
<td style="text-align:left;">
(0.032)
</td>
<td style="text-align:left;">
(0.049)
</td>
<td style="text-align:left;">
(0.129)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">wrkslf</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.854</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.899</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(71, 44, 122, 1) !important;">1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">1.871</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">2</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.941</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">1.917</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.891</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.921</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(71, 44, 122, 1) !important;">1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">1.806</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.857</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.863</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.866</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.893</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.923</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 125, 142, 1) !important;">1.689</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.862</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.922</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">1.944</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.879</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.941</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">2</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.026)
</td>
<td style="text-align:left;">
(0.027)
</td>
<td style="text-align:left;">
(0.009)
</td>
<td style="text-align:left;">
(0.009)
</td>
<td style="text-align:left;">
(0.039)
</td>
<td style="text-align:left;">
(0.056)
</td>
<td style="text-align:left;">
(0.024)
</td>
<td style="text-align:left;">
(0.008)
</td>
<td style="text-align:left;">
(0.031)
</td>
<td style="text-align:left;">
(0.034)
</td>
<td style="text-align:left;">
(0.019)
</td>
<td style="text-align:left;">
(0.021)
</td>
<td style="text-align:left;">
(0.02)
</td>
<td style="text-align:left;">
(0.052)
</td>
<td style="text-align:left;">
(0.015)
</td>
<td style="text-align:left;">
(0.035)
</td>
<td style="text-align:left;">
(0.028)
</td>
<td style="text-align:left;">
(0.021)
</td>
<td style="text-align:left;">
(0.065)
</td>
<td style="text-align:left;">
(0.017)
</td>
<td style="text-align:left;">
(0.025)
</td>
<td style="text-align:left;">
(0.067)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">marital</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 164, 134, 1) !important;">3.417</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.954</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.98</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(44, 115, 142, 1) !important;">1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">2.354</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">2.549</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 136, 1) !important;">3.25</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 172, 130, 1) !important;">3.806</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">2.559</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">1.852</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">3</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">2.968</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">2.065</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">2.532</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">2.714</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(41, 175, 127, 1) !important;">4.036</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.967</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">2.468</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">3.313</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">1.889</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">2.817</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(54, 183, 121, 1) !important;">4.441</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(56, 185, 119, 1) !important;">4.529</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.21)
</td>
<td style="text-align:left;">
(0.139)
</td>
<td style="text-align:left;">
(0.144)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.049)
</td>
<td style="text-align:left;">
(0.203)
</td>
<td style="text-align:left;">
(0.296)
</td>
<td style="text-align:left;">
(0.128)
</td>
<td style="text-align:left;">
(0.042)
</td>
<td style="text-align:left;">
(0.161)
</td>
<td style="text-align:left;">
(0.177)
</td>
<td style="text-align:left;">
(0.099)
</td>
<td style="text-align:left;">
(0.112)
</td>
<td style="text-align:left;">
(0.106)
</td>
<td style="text-align:left;">
(0.274)
</td>
<td style="text-align:left;">
(0.079)
</td>
<td style="text-align:left;">
(0.186)
</td>
<td style="text-align:left;">
(0.15)
</td>
<td style="text-align:left;">
(0.108)
</td>
<td style="text-align:left;">
(0.342)
</td>
<td style="text-align:left;">
(0.088)
</td>
<td style="text-align:left;">
(0.134)
</td>
<td style="text-align:left;">
(0.352)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">sibs</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 124, 142, 1) !important;">1.708</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 173, 129, 1) !important;">5.963</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">2.792</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">3.182</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">3.169</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 169, 131, 1) !important;">5.627</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(46, 179, 124, 1) !important;">6.458</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">3.395</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">3.702</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">3.049</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">4.269</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">3.171</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 156, 137, 1) !important;">4.435</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">3.446</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">3.25</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">3.552</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 158, 137, 1) !important;">4.623</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">2.649</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">3.302</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">4.667</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">2.846</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">3.356</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">3.765</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.418)
</td>
<td style="text-align:left;">
(0.277)
</td>
<td style="text-align:left;">
(0.288)
</td>
<td style="text-align:left;">
(0.095)
</td>
<td style="text-align:left;">
(0.098)
</td>
<td style="text-align:left;">
(0.405)
</td>
<td style="text-align:left;">
(0.591)
</td>
<td style="text-align:left;">
(0.255)
</td>
<td style="text-align:left;">
(0.085)
</td>
<td style="text-align:left;">
(0.322)
</td>
<td style="text-align:left;">
(0.354)
</td>
<td style="text-align:left;">
(0.196)
</td>
<td style="text-align:left;">
(0.223)
</td>
<td style="text-align:left;">
(0.212)
</td>
<td style="text-align:left;">
(0.547)
</td>
<td style="text-align:left;">
(0.158)
</td>
<td style="text-align:left;">
(0.37)
</td>
<td style="text-align:left;">
(0.298)
</td>
<td style="text-align:left;">
(0.216)
</td>
<td style="text-align:left;">
(0.682)
</td>
<td style="text-align:left;">
(0.175)
</td>
<td style="text-align:left;">
(0.266)
</td>
<td style="text-align:left;">
(0.702)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">childs</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.583</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 136, 1) !important;">2.303</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">1.911</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 154, 138, 1) !important;">2.025</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.442</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 165, 134, 1) !important;">2.471</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 165, 134, 1) !important;">2.5</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.38</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.656</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.778</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.209</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.171</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.798</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.495</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">1.071</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">1.264</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 158, 137, 1) !important;">2.18</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.213</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">1.006</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">2.111</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">1.106</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(41, 121, 142, 1) !important;">0.576</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">1</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.212)
</td>
<td style="text-align:left;">
(0.14)
</td>
<td style="text-align:left;">
(0.146)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.049)
</td>
<td style="text-align:left;">
(0.205)
</td>
<td style="text-align:left;">
(0.299)
</td>
<td style="text-align:left;">
(0.129)
</td>
<td style="text-align:left;">
(0.043)
</td>
<td style="text-align:left;">
(0.163)
</td>
<td style="text-align:left;">
(0.179)
</td>
<td style="text-align:left;">
(0.1)
</td>
<td style="text-align:left;">
(0.113)
</td>
<td style="text-align:left;">
(0.108)
</td>
<td style="text-align:left;">
(0.277)
</td>
<td style="text-align:left;">
(0.08)
</td>
<td style="text-align:left;">
(0.188)
</td>
<td style="text-align:left;">
(0.151)
</td>
<td style="text-align:left;">
(0.11)
</td>
<td style="text-align:left;">
(0.346)
</td>
<td style="text-align:left;">
(0.089)
</td>
<td style="text-align:left;">
(0.135)
</td>
<td style="text-align:left;">
(0.356)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">occ80</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(43, 117, 142, 1) !important;">141.2</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">358.1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">262</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">331.9</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">324</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">411</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(44, 115, 142, 1) !important;">123.6</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">327.5</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">323.1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 154, 138, 1) !important;">408</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(41, 175, 127, 1) !important;">562.4</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 129, 142, 1) !important;">222</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">339.2</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 162, 135, 1) !important;">463.5</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 160, 136, 1) !important;">442.3</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 167, 133, 1) !important;">496.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 173, 129, 1) !important;">540.9</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(50, 101, 142, 1) !important;">26.77</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">437.8</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 162, 135, 1) !important;">463.4</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">248.9</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(41, 121, 142, 1) !important;">170</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 165, 134, 1) !important;">479.7</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(32.76)
</td>
<td style="text-align:left;">
(21.74)
</td>
<td style="text-align:left;">
(22.58)
</td>
<td style="text-align:left;">
(7.475)
</td>
<td style="text-align:left;">
(7.647)
</td>
<td style="text-align:left;">
(31.78)
</td>
<td style="text-align:left;">
(46.33)
</td>
<td style="text-align:left;">
(19.98)
</td>
<td style="text-align:left;">
(6.635)
</td>
<td style="text-align:left;">
(25.22)
</td>
<td style="text-align:left;">
(27.73)
</td>
<td style="text-align:left;">
(15.41)
</td>
<td style="text-align:left;">
(17.51)
</td>
<td style="text-align:left;">
(16.64)
</td>
<td style="text-align:left;">
(42.89)
</td>
<td style="text-align:left;">
(12.36)
</td>
<td style="text-align:left;">
(29.06)
</td>
<td style="text-align:left;">
(23.41)
</td>
<td style="text-align:left;">
(16.96)
</td>
<td style="text-align:left;">
(53.5)
</td>
<td style="text-align:left;">
(13.74)
</td>
<td style="text-align:left;">
(20.89)
</td>
<td style="text-align:left;">
(55.05)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">prestg80</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(46, 180, 124, 1) !important;">59.98</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">43.8</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">48.3</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">44.94</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">47.83</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 127, 142, 1) !important;">38.02</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 174, 128, 1) !important;">57.83</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">46.34</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">45.14</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">42.49</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">39.39</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 171, 130, 1) !important;">56.02</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">44.89</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">41.65</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">44.5</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(44, 113, 142, 1) !important;">32.76</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">39.3</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 169, 131, 1) !important;">55.61</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(45, 112, 142, 1) !important;">31.89</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">44</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">50.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(46, 179, 124, 1) !important;">59.43</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(60, 79, 139, 1) !important;">19.88</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(1.791)
</td>
<td style="text-align:left;">
(1.189)
</td>
<td style="text-align:left;">
(1.235)
</td>
<td style="text-align:left;">
(0.409)
</td>
<td style="text-align:left;">
(0.418)
</td>
<td style="text-align:left;">
(1.738)
</td>
<td style="text-align:left;">
(2.533)
</td>
<td style="text-align:left;">
(1.093)
</td>
<td style="text-align:left;">
(0.363)
</td>
<td style="text-align:left;">
(1.379)
</td>
<td style="text-align:left;">
(1.516)
</td>
<td style="text-align:left;">
(0.842)
</td>
<td style="text-align:left;">
(0.957)
</td>
<td style="text-align:left;">
(0.91)
</td>
<td style="text-align:left;">
(2.345)
</td>
<td style="text-align:left;">
(0.676)
</td>
<td style="text-align:left;">
(1.589)
</td>
<td style="text-align:left;">
(1.28)
</td>
<td style="text-align:left;">
(0.927)
</td>
<td style="text-align:left;">
(2.925)
</td>
<td style="text-align:left;">
(0.751)
</td>
<td style="text-align:left;">
(1.142)
</td>
<td style="text-align:left;">
(3.01)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">indus80</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">722.1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">699.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">740.6</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">602.1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">562.3</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 154, 138, 1) !important;">686.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 163, 134, 1) !important;">756.3</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">629.2</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">674.5</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(56, 89, 140, 1) !important;">170.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(62, 76, 138, 1) !important;">82.27</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 168, 132, 1) !important;">794.4</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">598.8</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(51, 98, 141, 1) !important;">236.1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(53, 94, 141, 1) !important;">208.4</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">549.6</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(62, 76, 138, 1) !important;">78.13</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 165, 133, 1) !important;">772.3</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 159, 136, 1) !important;">717.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(62, 76, 138, 1) !important;">77.67</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">671.8</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 156, 137, 1) !important;">697.1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">603.9</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(33.38)
</td>
<td style="text-align:left;">
(22.15)
</td>
<td style="text-align:left;">
(23.01)
</td>
<td style="text-align:left;">
(7.617)
</td>
<td style="text-align:left;">
(7.792)
</td>
<td style="text-align:left;">
(32.39)
</td>
<td style="text-align:left;">
(47.21)
</td>
<td style="text-align:left;">
(20.36)
</td>
<td style="text-align:left;">
(6.762)
</td>
<td style="text-align:left;">
(25.7)
</td>
<td style="text-align:left;">
(28.26)
</td>
<td style="text-align:left;">
(15.7)
</td>
<td style="text-align:left;">
(17.84)
</td>
<td style="text-align:left;">
(16.96)
</td>
<td style="text-align:left;">
(43.71)
</td>
<td style="text-align:left;">
(12.6)
</td>
<td style="text-align:left;">
(29.61)
</td>
<td style="text-align:left;">
(23.86)
</td>
<td style="text-align:left;">
(17.29)
</td>
<td style="text-align:left;">
(54.52)
</td>
<td style="text-align:left;">
<ol start="14" style="list-style-type: decimal">
<li></li></ol></td>
<td style="text-align:left;">
(21.29)
</td>
<td style="text-align:left;">
(56.1)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">res16</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">3.583</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">3.44</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">3.564</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">3.332</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">3.421</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">3.098</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">3.667</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">3.558</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">3.662</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">3.111</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">2.955</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">3.737</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">3.946</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">3.419</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 156, 137, 1) !important;">4.071</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">3.353</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">3.328</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">4.043</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">3.709</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 129, 142, 1) !important;">2.833</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">4.092</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">3.585</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">3.706</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.218)
</td>
<td style="text-align:left;">
(0.145)
</td>
<td style="text-align:left;">
(0.15)
</td>
<td style="text-align:left;">
(0.05)
</td>
<td style="text-align:left;">
(0.051)
</td>
<td style="text-align:left;">
(0.212)
</td>
<td style="text-align:left;">
(0.308)
</td>
<td style="text-align:left;">
(0.133)
</td>
<td style="text-align:left;">
(0.044)
</td>
<td style="text-align:left;">
(0.168)
</td>
<td style="text-align:left;">
(0.184)
</td>
<td style="text-align:left;">
(0.102)
</td>
<td style="text-align:left;">
(0.116)
</td>
<td style="text-align:left;">
(0.111)
</td>
<td style="text-align:left;">
(0.285)
</td>
<td style="text-align:left;">
(0.082)
</td>
<td style="text-align:left;">
(0.193)
</td>
<td style="text-align:left;">
(0.156)
</td>
<td style="text-align:left;">
(0.113)
</td>
<td style="text-align:left;">
(0.356)
</td>
<td style="text-align:left;">
(0.091)
</td>
<td style="text-align:left;">
(0.139)
</td>
<td style="text-align:left;">
(0.366)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">reg16</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">4.104</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">4.156</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">4.604</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">4.718</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">4.766</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">4.216</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">4.792</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">4.907</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">4.471</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">4.642</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">4.716</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">3.839</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(57, 85, 140, 1) !important;">0</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">3.753</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(91, 200, 100, 1) !important;">9</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">4.935</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">4.393</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">3.766</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">4.369</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">5.389</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">4.681</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">4.568</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">4.647</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.356)
</td>
<td style="text-align:left;">
(0.236)
</td>
<td style="text-align:left;">
(0.246)
</td>
<td style="text-align:left;">
(0.081)
</td>
<td style="text-align:left;">
(0.083)
</td>
<td style="text-align:left;">
(0.346)
</td>
<td style="text-align:left;">
(0.504)
</td>
<td style="text-align:left;">
(0.217)
</td>
<td style="text-align:left;">
(0.072)
</td>
<td style="text-align:left;">
(0.274)
</td>
<td style="text-align:left;">
(0.302)
</td>
<td style="text-align:left;">
(0.168)
</td>
<td style="text-align:left;">
(0.19)
</td>
<td style="text-align:left;">
(0.181)
</td>
<td style="text-align:left;">
(0.467)
</td>
<td style="text-align:left;">
(0.134)
</td>
<td style="text-align:left;">
(0.316)
</td>
<td style="text-align:left;">
(0.255)
</td>
<td style="text-align:left;">
(0.184)
</td>
<td style="text-align:left;">
(0.582)
</td>
<td style="text-align:left;">
(0.149)
</td>
<td style="text-align:left;">
(0.227)
</td>
<td style="text-align:left;">
(0.599)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">mobile16</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">2.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">2.119</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.851</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">1.936</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.95</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">1.804</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.833</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">1.791</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.835</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.728</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 131, 142, 1) !important;">1.642</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">2.092</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(56, 185, 119, 1) !important;">3</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.984</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.821</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">1.665</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">2.049</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">2.117</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.855</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 129, 142, 1) !important;">1.556</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">2.125</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.831</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.824</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.119)
</td>
<td style="text-align:left;">
(0.079)
</td>
<td style="text-align:left;">
(0.082)
</td>
<td style="text-align:left;">
(0.027)
</td>
<td style="text-align:left;">
(0.028)
</td>
<td style="text-align:left;">
(0.116)
</td>
<td style="text-align:left;">
(0.168)
</td>
<td style="text-align:left;">
(0.073)
</td>
<td style="text-align:left;">
(0.024)
</td>
<td style="text-align:left;">
(0.092)
</td>
<td style="text-align:left;">
(0.101)
</td>
<td style="text-align:left;">
(0.056)
</td>
<td style="text-align:left;">
(0.064)
</td>
<td style="text-align:left;">
(0.061)
</td>
<td style="text-align:left;">
(0.156)
</td>
<td style="text-align:left;">
(0.045)
</td>
<td style="text-align:left;">
(0.106)
</td>
<td style="text-align:left;">
(0.085)
</td>
<td style="text-align:left;">
(0.062)
</td>
<td style="text-align:left;">
(0.195)
</td>
<td style="text-align:left;">
(0.05)
</td>
<td style="text-align:left;">
(0.076)
</td>
<td style="text-align:left;">
(0.2)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">family16</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">1.562</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.716</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.446</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.652</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.779</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 165, 134, 1) !important;">2.843</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.958</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.961</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.971</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.444</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.776</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.631</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.661</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.925</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">1.536</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.97</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.787</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">2.021</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.86</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.333</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.927</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.678</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 160, 136, 1) !important;">2.588</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.238)
</td>
<td style="text-align:left;">
(0.158)
</td>
<td style="text-align:left;">
(0.164)
</td>
<td style="text-align:left;">
(0.054)
</td>
<td style="text-align:left;">
(0.056)
</td>
<td style="text-align:left;">
(0.231)
</td>
<td style="text-align:left;">
(0.336)
</td>
<td style="text-align:left;">
(0.145)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.183)
</td>
<td style="text-align:left;">
(0.201)
</td>
<td style="text-align:left;">
(0.112)
</td>
<td style="text-align:left;">
(0.127)
</td>
<td style="text-align:left;">
(0.121)
</td>
<td style="text-align:left;">
(0.312)
</td>
<td style="text-align:left;">
(0.09)
</td>
<td style="text-align:left;">
(0.211)
</td>
<td style="text-align:left;">
(0.17)
</td>
<td style="text-align:left;">
(0.123)
</td>
<td style="text-align:left;">
(0.389)
</td>
<td style="text-align:left;">
(0.1)
</td>
<td style="text-align:left;">
(0.152)
</td>
<td style="text-align:left;">
(0.4)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">parborn</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">0.833</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(46, 180, 124, 1) !important;">3.514</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">0.465</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">0.409</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">0.358</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">2.216</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 131, 142, 1) !important;">0.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">0.574</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">0.695</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">0.383</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">0.567</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">0.935</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(154, 217, 60, 1) !important;">7.006</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.027</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.321</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">0.466</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.082</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">1.447</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">0.95</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.333</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">0.7</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">0.144</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">0.059</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.299)
</td>
<td style="text-align:left;">
(0.198)
</td>
<td style="text-align:left;">
(0.206)
</td>
<td style="text-align:left;">
(0.068)
</td>
<td style="text-align:left;">
(0.07)
</td>
<td style="text-align:left;">
(0.29)
</td>
<td style="text-align:left;">
(0.423)
</td>
<td style="text-align:left;">
(0.182)
</td>
<td style="text-align:left;">
(0.061)
</td>
<td style="text-align:left;">
(0.23)
</td>
<td style="text-align:left;">
(0.253)
</td>
<td style="text-align:left;">
(0.141)
</td>
<td style="text-align:left;">
(0.16)
</td>
<td style="text-align:left;">
(0.152)
</td>
<td style="text-align:left;">
(0.392)
</td>
<td style="text-align:left;">
(0.113)
</td>
<td style="text-align:left;">
(0.265)
</td>
<td style="text-align:left;">
(0.214)
</td>
<td style="text-align:left;">
(0.155)
</td>
<td style="text-align:left;">
(0.488)
</td>
<td style="text-align:left;">
(0.125)
</td>
<td style="text-align:left;">
(0.191)
</td>
<td style="text-align:left;">
(0.503)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">maeduc</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">11.19</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(65, 67, 135, 1) !important;">4.615</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">12.35</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">11.72</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">12.2</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(66, 64, 134, 1) !important;">4.392</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">10.63</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">12.38</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">11.45</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">11.54</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 131, 142, 1) !important;">10.51</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">13.01</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 120, 142, 1) !important;">9.333</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">11.48</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">12.39</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">11.57</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 126, 142, 1) !important;">9.918</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">12.8</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">11.9</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 123, 142, 1) !important;">9.722</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(70, 192, 111, 1) !important;">16.56</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 136, 1) !important;">13.28</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">11.06</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.397)
</td>
<td style="text-align:left;">
(0.264)
</td>
<td style="text-align:left;">
(0.274)
</td>
<td style="text-align:left;">
(0.091)
</td>
<td style="text-align:left;">
(0.093)
</td>
<td style="text-align:left;">
(0.385)
</td>
<td style="text-align:left;">
(0.562)
</td>
<td style="text-align:left;">
(0.242)
</td>
<td style="text-align:left;">
(0.08)
</td>
<td style="text-align:left;">
(0.306)
</td>
<td style="text-align:left;">
(0.336)
</td>
<td style="text-align:left;">
(0.187)
</td>
<td style="text-align:left;">
(0.212)
</td>
<td style="text-align:left;">
(0.202)
</td>
<td style="text-align:left;">
(0.52)
</td>
<td style="text-align:left;">
(0.15)
</td>
<td style="text-align:left;">
(0.352)
</td>
<td style="text-align:left;">
(0.284)
</td>
<td style="text-align:left;">
(0.206)
</td>
<td style="text-align:left;">
(0.648)
</td>
<td style="text-align:left;">
(0.166)
</td>
<td style="text-align:left;">
(0.253)
</td>
<td style="text-align:left;">
(0.667)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">degree</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 163, 134, 1) !important;">2.417</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">1.541</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">2.178</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.629</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.924</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(41, 121, 142, 1) !important;">0.961</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.583</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.457</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.714</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">1.556</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 126, 142, 1) !important;">1.104</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">2.359</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.851</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.441</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.643</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 126, 142, 1) !important;">1.116</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 126, 142, 1) !important;">1.098</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 169, 131, 1) !important;">2.596</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 128, 142, 1) !important;">1.156</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">1.278</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 167, 133, 1) !important;">2.542</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">2.076</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">1.294</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.16)
</td>
<td style="text-align:left;">
(0.106)
</td>
<td style="text-align:left;">
(0.11)
</td>
<td style="text-align:left;">
(0.036)
</td>
<td style="text-align:left;">
(0.037)
</td>
<td style="text-align:left;">
(0.155)
</td>
<td style="text-align:left;">
(0.226)
</td>
<td style="text-align:left;">
(0.097)
</td>
<td style="text-align:left;">
(0.032)
</td>
<td style="text-align:left;">
(0.123)
</td>
<td style="text-align:left;">
(0.135)
</td>
<td style="text-align:left;">
(0.075)
</td>
<td style="text-align:left;">
(0.085)
</td>
<td style="text-align:left;">
(0.081)
</td>
<td style="text-align:left;">
(0.209)
</td>
<td style="text-align:left;">
(0.06)
</td>
<td style="text-align:left;">
(0.142)
</td>
<td style="text-align:left;">
(0.114)
</td>
<td style="text-align:left;">
(0.083)
</td>
<td style="text-align:left;">
(0.26)
</td>
<td style="text-align:left;">
(0.067)
</td>
<td style="text-align:left;">
(0.102)
</td>
<td style="text-align:left;">
(0.268)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">sex</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 158, 137, 1) !important;">1.708</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.596</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 129, 142, 1) !important;">1.277</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.569</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">1.301</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">1.608</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 155, 138, 1) !important;">1.667</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">1.612</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">1.62</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 120, 142, 1) !important;">1.148</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.373</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">1.622</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.47</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">1.328</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">1.286</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">1.558</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 128, 142, 1) !important;">1.262</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.351</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 149, 139, 1) !important;">1.57</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 129, 142, 1) !important;">1.278</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.524</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.517</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.529</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.07)
</td>
<td style="text-align:left;">
(0.046)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.016)
</td>
<td style="text-align:left;">
(0.016)
</td>
<td style="text-align:left;">
(0.068)
</td>
<td style="text-align:left;">
(0.098)
</td>
<td style="text-align:left;">
(0.042)
</td>
<td style="text-align:left;">
(0.014)
</td>
<td style="text-align:left;">
(0.054)
</td>
<td style="text-align:left;">
(0.059)
</td>
<td style="text-align:left;">
(0.033)
</td>
<td style="text-align:left;">
(0.037)
</td>
<td style="text-align:left;">
(0.035)
</td>
<td style="text-align:left;">
(0.091)
</td>
<td style="text-align:left;">
(0.026)
</td>
<td style="text-align:left;">
(0.062)
</td>
<td style="text-align:left;">
(0.05)
</td>
<td style="text-align:left;">
(0.036)
</td>
<td style="text-align:left;">
(0.114)
</td>
<td style="text-align:left;">
(0.029)
</td>
<td style="text-align:left;">
(0.044)
</td>
<td style="text-align:left;">
(0.117)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">race</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">1.062</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 171, 130, 1) !important;">1.697</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.089</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.095</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.11</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 156, 137, 1) !important;">1.451</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.167</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.109</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 151, 139, 1) !important;">1.368</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.123</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">1.463</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.221</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(51, 182, 121, 1) !important;">1.899</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.204</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.179</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.184</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.344</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.191</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.279</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 173, 130, 1) !important;">1.722</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.293</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.169</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.118</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.077)
</td>
<td style="text-align:left;">
(0.051)
</td>
<td style="text-align:left;">
(0.053)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.074)
</td>
<td style="text-align:left;">
(0.108)
</td>
<td style="text-align:left;">
(0.047)
</td>
<td style="text-align:left;">
(0.016)
</td>
<td style="text-align:left;">
(0.059)
</td>
<td style="text-align:left;">
(0.065)
</td>
<td style="text-align:left;">
(0.036)
</td>
<td style="text-align:left;">
(0.041)
</td>
<td style="text-align:left;">
(0.039)
</td>
<td style="text-align:left;">
(0.1)
</td>
<td style="text-align:left;">
(0.029)
</td>
<td style="text-align:left;">
(0.068)
</td>
<td style="text-align:left;">
(0.055)
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.125)
</td>
<td style="text-align:left;">
(0.032)
</td>
<td style="text-align:left;">
(0.049)
</td>
<td style="text-align:left;">
(0.129)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">born</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">1.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 173, 129, 1) !important;">1.349</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.05</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">1.028</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 136, 142, 1) !important;">1.024</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">1.196</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">1</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">1.047</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.059</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">1.012</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">1.06</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.097</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(187, 223, 39, 1) !important;">1.911</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">1.118</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 158, 137, 1) !important;">1.214</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">1.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">1.098</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">1.138</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">1.089</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">1.167</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.084</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">1.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">1</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.034)
</td>
<td style="text-align:left;">
(0.023)
</td>
<td style="text-align:left;">
(0.023)
</td>
<td style="text-align:left;">
(0.008)
</td>
<td style="text-align:left;">
(0.008)
</td>
<td style="text-align:left;">
(0.033)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.021)
</td>
<td style="text-align:left;">
(0.007)
</td>
<td style="text-align:left;">
(0.026)
</td>
<td style="text-align:left;">
(0.029)
</td>
<td style="text-align:left;">
(0.016)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.017)
</td>
<td style="text-align:left;">
(0.044)
</td>
<td style="text-align:left;">
(0.013)
</td>
<td style="text-align:left;">
(0.03)
</td>
<td style="text-align:left;">
(0.024)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.055)
</td>
<td style="text-align:left;">
(0.014)
</td>
<td style="text-align:left;">
(0.022)
</td>
<td style="text-align:left;">
(0.057)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">hompop</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(42, 118, 142, 1) !important;">1.583</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 154, 138, 1) !important;">3.073</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">2.653</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 157, 137, 1) !important;">3.198</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">2.608</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">2.765</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(43, 116, 142, 1) !important;">1.5</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 127, 142, 1) !important;">1.922</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">2.595</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">2.802</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">2.746</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">2.401</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">2.893</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">2.629</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">2.714</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">2.237</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">2.656</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">2.415</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">2.508</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(56, 185, 119, 1) !important;">4.389</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">2.484</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">2.068</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">2.118</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.197)
</td>
<td style="text-align:left;">
(0.131)
</td>
<td style="text-align:left;">
(0.136)
</td>
<td style="text-align:left;">
(0.045)
</td>
<td style="text-align:left;">
(0.046)
</td>
<td style="text-align:left;">
(0.192)
</td>
<td style="text-align:left;">
(0.279)
</td>
<td style="text-align:left;">
(0.12)
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.152)
</td>
<td style="text-align:left;">
(0.167)
</td>
<td style="text-align:left;">
(0.093)
</td>
<td style="text-align:left;">
(0.106)
</td>
<td style="text-align:left;">
(0.1)
</td>
<td style="text-align:left;">
(0.258)
</td>
<td style="text-align:left;">
(0.074)
</td>
<td style="text-align:left;">
(0.175)
</td>
<td style="text-align:left;">
(0.141)
</td>
<td style="text-align:left;">
(0.102)
</td>
<td style="text-align:left;">
(0.322)
</td>
<td style="text-align:left;">
(0.083)
</td>
<td style="text-align:left;">
(0.126)
</td>
<td style="text-align:left;">
(0.332)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">babies</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">0</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">0.22</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">0.129</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">0.357</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">0.218</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">0.294</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 132, 142, 1) !important;">0.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">0.07</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">0.227</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">0.235</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">0</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">0.212</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">0.214</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">0.172</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">0.214</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">0.16</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">0.066</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">0.138</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">0.257</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(122, 209, 81, 1) !important;">1.389</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">0.231</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">0.203</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">0.353</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.078)
</td>
<td style="text-align:left;">
(0.052)
</td>
<td style="text-align:left;">
(0.054)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.076)
</td>
<td style="text-align:left;">
(0.111)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.016)
</td>
<td style="text-align:left;">
(0.06)
</td>
<td style="text-align:left;">
(0.066)
</td>
<td style="text-align:left;">
(0.037)
</td>
<td style="text-align:left;">
(0.042)
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.102)
</td>
<td style="text-align:left;">
(0.03)
</td>
<td style="text-align:left;">
(0.069)
</td>
<td style="text-align:left;">
(0.056)
</td>
<td style="text-align:left;">
(0.041)
</td>
<td style="text-align:left;">
(0.128)
</td>
<td style="text-align:left;">
(0.033)
</td>
<td style="text-align:left;">
(0.05)
</td>
<td style="text-align:left;">
(0.132)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">preteen</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">0.125</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">0.312</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">0.396</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">0.42</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">0.328</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">0.314</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(38, 130, 142, 1) !important;">0.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 137, 142, 1) !important;">0.17</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">0.294</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">0.284</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 153, 138, 1) !important;">0.492</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">0.207</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">0.339</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">0.21</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">0.214</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">0.223</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">0.295</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">0.266</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">0.179</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 162, 135, 1) !important;">0.667</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">0.26</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">0.22</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 131, 142, 1) !important;">0.059</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.095)
</td>
<td style="text-align:left;">
(0.063)
</td>
<td style="text-align:left;">
(0.065)
</td>
<td style="text-align:left;">
(0.022)
</td>
<td style="text-align:left;">
(0.022)
</td>
<td style="text-align:left;">
(0.092)
</td>
<td style="text-align:left;">
(0.134)
</td>
<td style="text-align:left;">
(0.058)
</td>
<td style="text-align:left;">
(0.019)
</td>
<td style="text-align:left;">
(0.073)
</td>
<td style="text-align:left;">
(0.08)
</td>
<td style="text-align:left;">
(0.044)
</td>
<td style="text-align:left;">
(0.051)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.124)
</td>
<td style="text-align:left;">
(0.036)
</td>
<td style="text-align:left;">
(0.084)
</td>
<td style="text-align:left;">
(0.068)
</td>
<td style="text-align:left;">
(0.049)
</td>
<td style="text-align:left;">
(0.154)
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.06)
</td>
<td style="text-align:left;">
(0.159)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">teens</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">0.042</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">0.266</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">0.257</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">0.25</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">0.196</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 154, 138, 1) !important;">0.372</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">0.25</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">0.155</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 144, 141, 1) !important;">0.213</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">0.259</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 148, 140, 1) !important;">0.284</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">0.161</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">0.268</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 145, 140, 1) !important;">0.226</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">0.321</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 140, 141, 1) !important;">0.157</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">0.262</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">0.17</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 138, 141, 1) !important;">0.117</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">0.056</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">0.172</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(35, 139, 141, 1) !important;">0.127</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 134, 142, 1) !important;">0.059</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.075)
</td>
<td style="text-align:left;">
(0.05)
</td>
<td style="text-align:left;">
(0.052)
</td>
<td style="text-align:left;">
(0.017)
</td>
<td style="text-align:left;">
(0.018)
</td>
<td style="text-align:left;">
(0.073)
</td>
<td style="text-align:left;">
(0.106)
</td>
<td style="text-align:left;">
(0.046)
</td>
<td style="text-align:left;">
(0.015)
</td>
<td style="text-align:left;">
(0.058)
</td>
<td style="text-align:left;">
(0.064)
</td>
<td style="text-align:left;">
(0.035)
</td>
<td style="text-align:left;">
(0.04)
</td>
<td style="text-align:left;">
(0.038)
</td>
<td style="text-align:left;">
(0.099)
</td>
<td style="text-align:left;">
(0.028)
</td>
<td style="text-align:left;">
(0.067)
</td>
<td style="text-align:left;">
(0.054)
</td>
<td style="text-align:left;">
(0.039)
</td>
<td style="text-align:left;">
(0.123)
</td>
<td style="text-align:left;">
(0.032)
</td>
<td style="text-align:left;">
(0.048)
</td>
<td style="text-align:left;">
(0.126)
</td>
</tr>
<tr>
<td style="text-align:left;">
<span style=" font-weight: bold;    color: black !important;">adults</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(40, 123, 142, 1) !important;">1.417</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 136, 1) !important;">2.257</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 143, 141, 1) !important;">1.871</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(30, 156, 137, 1) !important;">2.153</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.857</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 139, 141, 1) !important;">1.784</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(45, 113, 142, 1) !important;">1.167</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 128, 142, 1) !important;">1.527</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.856</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">1.975</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">1.97</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(34, 141, 141, 1) !important;">1.82</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 152, 139, 1) !important;">2.071</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">2.022</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 147, 140, 1) !important;">1.964</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(36, 135, 142, 1) !important;">1.697</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 150, 139, 1) !important;">2.033</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.84</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(32, 146, 140, 1) !important;">1.955</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(31, 161, 135, 1) !important;">2.278</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(33, 142, 141, 1) !important;">1.828</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(39, 128, 142, 1) !important;">1.517</span>
</td>
<td style="text-align:left;">
<span style="     color: White !important;border-radius: 4px; padding-right: 4px; padding-left: 4px; background-color: rgba(37, 133, 142, 1) !important;">1.647</span>
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:left;">
(0.109)
</td>
<td style="text-align:left;">
(0.073)
</td>
<td style="text-align:left;">
(0.075)
</td>
<td style="text-align:left;">
(0.025)
</td>
<td style="text-align:left;">
(0.026)
</td>
<td style="text-align:left;">
(0.106)
</td>
<td style="text-align:left;">
(0.155)
</td>
<td style="text-align:left;">
(0.067)
</td>
<td style="text-align:left;">
(0.022)
</td>
<td style="text-align:left;">
(0.084)
</td>
<td style="text-align:left;">
(0.093)
</td>
<td style="text-align:left;">
(0.051)
</td>
<td style="text-align:left;">
(0.058)
</td>
<td style="text-align:left;">
(0.056)
</td>
<td style="text-align:left;">
(0.143)
</td>
<td style="text-align:left;">
(0.041)
</td>
<td style="text-align:left;">
(0.097)
</td>
<td style="text-align:left;">
(0.078)
</td>
<td style="text-align:left;">
(0.057)
</td>
<td style="text-align:left;">
(0.178)
</td>
<td style="text-align:left;">
(0.046)
</td>
<td style="text-align:left;">
(0.07)
</td>
<td style="text-align:left;">
(0.184)
</td>
</tr>
</tbody>
<tfoot>
<tr>
<td style="padding: 0; " colspan="100%">
<span style="font-style: italic;">Note: </span>
</td>
</tr>
<tr>
<td style="padding: 0; " colspan="100%">
<sup></sup> Colors are assigned according to where the subgroup’s mean value lands on the standardized empirical distribution of it’s variable: (x - mean(x))/sd(x)<br>Standardized distribution is colored from a scale of +/-2.831
</td>
</tr>
</tfoot>


We can also estimate the following normalized measure of variation across leaves. A higher number indicates that there is higher variation in across leaf averages.

$$
\frac{Var \left( E[X_{i} | L_{i}] \right)}{Var(X_{i})}
$$

In [ ]:
# Adding a factor column turns all columns into character
df_est <- as.data.frame(apply(df_est, 2, as.numeric))

In [ ]:

covariate_means_per_leaf <- aggregate(. ~ leaf, df_est, mean)[,covariate_names]
covariate_means <- apply(df_est, 2, mean)[covariate_names]
leaf_weights <- table(df_est$leaf) / dim(df_est)[1] 
deviations <- t(apply(covariate_means_per_leaf, 1, function(x) x - covariate_means))
covariate_means_weighted_var <- apply(deviations, 2, function(x) sum(leaf_weights * x^2))
covariate_var <- apply(df_est, 2, var)[covariate_names]
cov_variation <- covariate_means_weighted_var / covariate_var



<caption>
Covariate variation across leaves
</caption>
<tbody>
<tr>
<td style="text-align:left;">
partyid
</td>
<td style="text-align:right;">
0.6692
</td>
</tr>
<tr>
<td style="text-align:left;">
polviews
</td>
<td style="text-align:right;">
0.4340
</td>
</tr>
<tr>
<td style="text-align:left;">
born
</td>
<td style="text-align:right;">
0.3140
</td>
</tr>
<tr>
<td style="text-align:left;">
maeduc
</td>
<td style="text-align:right;">
0.3016
</td>
</tr>
<tr>
<td style="text-align:left;">
indus80
</td>
<td style="text-align:right;">
0.2862
</td>
</tr>
<tr>
<td style="text-align:left;">
wrkslf
</td>
<td style="text-align:right;">
0.2862
</td>
</tr>
<tr>
<td style="text-align:left;">
wrkslf.1
</td>
<td style="text-align:right;">
0.2862
</td>
</tr>
<tr>
<td style="text-align:left;">
hrs1
</td>
<td style="text-align:right;">
0.2822
</td>
</tr>
<tr>
<td style="text-align:left;">
marital
</td>
<td style="text-align:right;">
0.2662
</td>
</tr>
<tr>
<td style="text-align:left;">
parborn
</td>
<td style="text-align:right;">
0.2592
</td>
</tr>
<tr>
<td style="text-align:left;">
prestg80
</td>
<td style="text-align:right;">
0.1935
</td>
</tr>
<tr>
<td style="text-align:left;">
occ80
</td>
<td style="text-align:right;">
0.1349
</td>
</tr>
<tr>
<td style="text-align:left;">
reg16
</td>
<td style="text-align:right;">
0.1218
</td>
</tr>
<tr>
<td style="text-align:left;">
degree
</td>
<td style="text-align:right;">
0.1038
</td>
</tr>
<tr>
<td style="text-align:left;">
educ
</td>
<td style="text-align:right;">
0.1029
</td>
</tr>
<tr>
<td style="text-align:left;">
race
</td>
<td style="text-align:right;">
0.1029
</td>
</tr>
<tr>
<td style="text-align:left;">
race.1
</td>
<td style="text-align:right;">
0.1029
</td>
</tr>
<tr>
<td style="text-align:left;">
income
</td>
<td style="text-align:right;">
0.0968
</td>
</tr>
<tr>
<td style="text-align:left;">
rincome
</td>
<td style="text-align:right;">
0.0842
</td>
</tr>
<tr>
<td style="text-align:left;">
age
</td>
<td style="text-align:right;">
0.0817
</td>
</tr>
<tr>
<td style="text-align:left;">
sex
</td>
<td style="text-align:right;">
0.0735
</td>
</tr>
<tr>
<td style="text-align:left;">
mobile16
</td>
<td style="text-align:right;">
0.0701
</td>
</tr>
<tr>
<td style="text-align:left;">
earnrs
</td>
<td style="text-align:right;">
0.0687
</td>
</tr>
<tr>
<td style="text-align:left;">
wrkstat
</td>
<td style="text-align:right;">
0.0630
</td>
</tr>
<tr>
<td style="text-align:left;">
hompop
</td>
<td style="text-align:right;">
0.0626
</td>
</tr>
<tr>
<td style="text-align:left;">
childs
</td>
<td style="text-align:right;">
0.0602
</td>
</tr>
<tr>
<td style="text-align:left;">
adults
</td>
<td style="text-align:right;">
0.0527
</td>
</tr>
<tr>
<td style="text-align:left;">
sibs
</td>
<td style="text-align:right;">
0.0448
</td>
</tr>
<tr>
<td style="text-align:left;">
babies
</td>
<td style="text-align:right;">
0.0349
</td>
</tr>
<tr>
<td style="text-align:left;">
res16
</td>
<td style="text-align:right;">
0.0246
</td>
</tr>
<tr>
<td style="text-align:left;">
preteen
</td>
<td style="text-align:right;">
0.0156
</td>
</tr>
<tr>
<td style="text-align:left;">
family16
</td>
<td style="text-align:right;">
0.0130
</td>
</tr>
<tr>
<td style="text-align:left;">
teens
</td>
<td style="text-align:right;">
0.0085
</td>
</tr>
</tbody>


----